In [1]:
import os
%mkdir "train/dog"
%mkdir "train/cat"

mkdir: cannot create directory ‘train/dog’: File exists
mkdir: cannot create directory ‘train/cat’: File exists


In [2]:
for dir, subdir, files in os.walk("train"):
    if len(subdir) == 0:
        continue
    for file in files:
        category = file.split(".")[0]
        os.rename("{}/{}".format(dir,file), "{}/{}/{}".format(dir, category, file))

In [ ]:
%mkdir "valid"
%mkdir "valid/dog"
%mkdir "valid/cat"

In [ ]:
dogs = [x for x in os.listdir("train/dog")]
cats = [x for x in os.listdir("train/cat")]


In [ ]:
import random
if len(os.listdir("valid/dog")) < 1:
    for n in random.sample(range(len(dogs)), 1000):
        os.rename("train/dog/{}".format(dogs[n]), "valid/dog/{}".format(dogs[n]))

if len(os.listdir("valid/cat")) < 1:
    for n in random.sample(range(len(cats)), 1000):
        os.rename("train/cat/{}".format(cats[n]), "valid/cat/{}".format(cats[n]))

In [1]:
ROWS = 128
COLS = 128
CHANNELS = 3
BATCH_SIZE=32

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(ROWS,COLS),
        batch_size=BATCH_SIZE,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'valid',
        target_size=(ROWS, COLS),
        batch_size=BATCH_SIZE,
        class_mode='binary')

Using TensorFlow backend.


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling2D
from keras.regularizers import l2

model = Sequential()
model.add(Convolution2D(16, 4, 4, border_mode='same', input_shape=(ROWS,COLS,CHANNELS),activation='relu'))
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), border_mode='same'))
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dense(output_dim=64, W_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(output_dim=1, W_regularizer=l2(0.01)))  #binary classification
model.add(Activation('relu'))

In [3]:
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=1e-4), metrics=['accuracy'])
model.fit_generator(generator=train_generator,samples_per_epoch=160,nb_epoch=20)

Epoch 1/20
160/160 [==============================] - 2s - loss: 3.0766 - acc: 0.5062     
Epoch 2/20
160/160 [==============================] - 0s - loss: 1.9629 - acc: 0.5687     
Epoch 3/20
160/160 [==============================] - 0s - loss: 2.2736 - acc: 0.6000     
Epoch 4/20
160/160 [==============================] - 0s - loss: 2.0710 - acc: 0.5062     
Epoch 5/20
160/160 [==============================] - 0s - loss: 1.9150 - acc: 0.5062     
Epoch 6/20
160/160 [==============================] - 0s - loss: 1.9461 - acc: 0.5500     
Epoch 7/20
160/160 [==============================] - 0s - loss: 1.8470 - acc: 0.4937     
Epoch 8/20
160/160 [==============================] - 0s - loss: 1.9715 - acc: 0.6063     
Epoch 9/20
160/160 [==============================] - 0s - loss: 1.7651 - acc: 0.5438     
Epoch 10/20
160/160 [==============================] - 0s - loss: 2.0107 - acc: 0.4875     
Epoch 11/20
160/160 [==============================] - 0s - loss: 1.7785 - acc: 0.5250   

In [ ]:
print("valid set :", model.evaluate_generator(validation_generator,val_samples=2000)[1]*100, "%")
print("--------------------")
print("train set :", model.evaluate_generator(train_generator,val_samples=2000)[1]*100, "%")


In [ ]:
model.evaluate_generator(train_generator,val_samples=2000)